In [6]:
import numpy as np
import itertools
from itertools import combinations
from multiprocessing import get_context
from typing import Optional, Tuple, List
from path_helpers import process_edge  # Ensure this is correctly implemented

def solve_hamiltonian_path_dp(distance_matrix: np.ndarray) -> Tuple[Optional[List[int]], float]:
    """
    Solve the Hamiltonian Path problem using dynamic programming.
    Time complexity: O(n^2 * 2^n)
    Space complexity: O(n * 2^n)
    
    Args:
        distance_matrix: n x n matrix of distances between points
        
    Returns:
        tuple: (optimal path, optimal distance) or (None, float('inf')) if no path exists
    """
    n = len(distance_matrix)
    # dp[mask][end] represents the minimum distance to visit all vertices in mask
    # and end at vertex 'end'
    dp = {}
    # prev[mask][end] stores the previous vertex in the optimal path
    prev = {}
    
    # Initialize dp array for single vertices
    for i in range(n):
        mask = 1 << i  # Create a mask with only vertex i
        dp[(mask, i)] = 0
    
    # Iterate through all possible subsets of vertices
    for size in range(2, n + 1):
        for subset in combinations(range(n), size):
            # Convert subset to bit mask
            mask = sum(1 << i for i in subset)
            
            # Try each vertex as the end point
            for end in subset:
                # Remove end point from consideration
                prev_mask = mask ^ (1 << end)
                min_dist = float('inf')
                best_prev = None
                
                # Try each vertex as the previous point
                for prev_end in subset:
                    if prev_end != end:
                        # Check if we can improve current solution
                        curr_dist = dp.get((prev_mask, prev_end), float('inf'))
                        if curr_dist != float('inf'):
                            total_dist = curr_dist + distance_matrix[prev_end][end]
                            if total_dist < min_dist:
                                min_dist = total_dist
                                best_prev = prev_end
                
                if min_dist != float('inf'):
                    dp[(mask, end)] = min_dist
                    prev[(mask, end)] = best_prev
    
    # Find the optimal solution
    final_mask = (1 << n) - 1  # All vertices visited
    min_total_dist = float('inf')
    best_end = None
    
    for end in range(n):
        if (final_mask, end) in dp and dp[(final_mask, end)] < min_total_dist:
            min_total_dist = dp[(final_mask, end)]
            best_end = end
    
    if best_end is None:
        return None, float('inf')
    
    # Reconstruct the path
    path = []
    current_end = best_end
    current_mask = final_mask
    
    while current_mask:
        path.append(current_end)
        next_mask = current_mask ^ (1 << current_end)
        current_end = prev.get((current_mask, current_end))
        current_mask = next_mask
        
        if current_end is None:
            break
    
    return list(reversed(path)), min_total_dist

def benchmark_hamiltonian_path(n: int = 12, instances: int = 10, seed: int = 42) -> None:
    """
    Benchmark the custom parallel Hamiltonian Path algorithm against the DP solution.
    
    Args:
        n: Number of points in each instance.
        instances: Number of instances to benchmark.
        seed: Base seed for random number generation.
        
    Returns:
        None
    """
    custom_distances = []
    dp_distances = []
    
    print(f"Starting Benchmark: {instances} instances with n={n} points each.\n")
    
    for instance in range(1, instances + 1):
        current_seed = seed + instance  # Different seed for each instance
        np.random.seed(current_seed)
        points = np.random.rand(n, 2)
        
        # Compute distance matrix
        distance_matrix = np.zeros((n, n))
        for i in range(n):
            for j in range(n):
                if i != j:
                    distance_matrix[i][j] = np.linalg.norm(points[i] - points[j])
        
        edges = list(itertools.combinations(range(n), 2))
        print(f"Instance {instance}: Processing {len(edges)} edges...")
        
        # Prepare arguments for multiprocessing
        args_list = [(edge, n, distance_matrix) for edge in edges]
        
        # Use multiprocessing with 'spawn' start method
        ctx = get_context('spawn')
        with ctx.Pool() as pool:
            results = pool.map(process_edge, args_list)
        
        # Filter out None results and find the best path
        complete_paths = [result for result in results if result is not None]
        
        if not complete_paths:
            print(f"Instance {instance}: No complete path found by the custom algorithm.\n")
            break  # Exit benchmarking as per instructions
        
        # Find the best path from custom algorithm
        best_result = min(complete_paths, key=lambda x: x[0])
        best_distance, best_path, _ = best_result
        custom_distances.append(best_distance)
        
        # Solve using Dynamic Programming
        dp_path, dp_distance = solve_hamiltonian_path_dp(distance_matrix)
        if dp_path is None:
            print(f"Instance {instance}: No complete path found by DP.\n")
            break  # Exit benchmarking as per instructions
        dp_distances.append(dp_distance)
        
        # Compare distances
        if best_distance > dp_distance:
            print(f"Instance {instance}: Custom distance {best_distance:.4f} > DP distance {dp_distance:.4f}.")
            print("Benchmark terminated: Custom algorithm found a worse solution than DP.")
            break  # Exit benchmarking as per instructions
        else:
            print(f"Instance {instance}: Custom distance {best_distance:.4f} <= DP distance {dp_distance:.4f}.\n")
    
    # Summary of Benchmarking
    total_run = len(custom_distances)
    if total_run == 0:
        print("No instances were successfully benchmarked.")
        return
    
    print("Benchmarking Summary:")
    print(f"Total Instances Run: {total_run} out of {instances}")
    print(f"Custom Algorithm - Average Distance: {np.mean(custom_distances):.4f}")
    print(f"DP Solution       - Average Distance: {np.mean(dp_distances):.4f}")
    improvement_percent = ((np.mean(dp_distances) - np.mean(custom_distances)) / np.mean(dp_distances)) * 100
    print(f"Average Improvement by Custom Algorithm: {improvement_percent:.2f}%")




In [7]:
benchmark_hamiltonian_path(n=12, instances=10, seed=42)

Starting Benchmark: 10 instances with n=12 points each.

Instance 1: Processing 66 edges...
Instance 1: Custom distance 2.8383 <= DP distance 2.8383.

Instance 2: Processing 66 edges...
Instance 2: Custom distance 2.5451 <= DP distance 2.5451.

Instance 3: Processing 66 edges...
Instance 3: Custom distance 2.3638 <= DP distance 2.3638.

Instance 4: Processing 66 edges...
Instance 4: Custom distance 2.3754 <= DP distance 2.3754.

Instance 5: Processing 66 edges...
Instance 5: Custom distance 2.7414 <= DP distance 2.7414.

Instance 6: Processing 66 edges...
Instance 6: Custom distance 2.3061 <= DP distance 2.3061.

Instance 7: Processing 66 edges...
Instance 7: Custom distance 2.5184 <= DP distance 2.5184.

Instance 8: Processing 66 edges...
Instance 8: Custom distance 2.0260 <= DP distance 2.0260.

Instance 9: Processing 66 edges...
Instance 9: Custom distance 2.3420 <= DP distance 2.3420.

Instance 10: Processing 66 edges...
Instance 10: Custom distance 2.7354 <= DP distance 2.7354.

B